In [ ]:
import os
import glob
import numpy as np
import csv
import random
from collections import OrderedDict
from six import raise_from

In [1]:
#Mount google drive to colab
from google.colab import drive
drive.mount('/content/gdrive')

MessageError: ignored

In [ ]:
data_path = "./gdrive/MyDrive/data/"

In [ ]:
def parse(value, function, fmt):
    """
    Parse a string into a value, and format a nice ValueError if it fails.

    Returns `function(value)`.
    Any `ValueError` raised is catched and a new `ValueError` is raised
    with message `fmt.format(e)`, where `e` is the caught `ValueError`.
    """
    try:
        return function(value)
    except ValueError as e:
        raise_from(ValueError(fmt.format(e)), None)

def read_annotations(csv_reader, classes=["defect"]):
    """
    Read annotations from the csv_reader.
    Args:
        csv_reader: csv reader of args.annotations_path
        classes: list[str] all the class names read from args.classes_path

    Returns:
        result: dict, dict is like {image_path: [{'x1': x1, 'y1': y1, 'x2': x2, 'y2': y2, 'class': class_name]}

    """
    result = OrderedDict()
    for line, row in enumerate(csv_reader, 1):
        try:
            img_file, x1, y1, x2, y2, class_name = row[:6]
            if class_name == 'defect':
              class_name = '1'
            if img_file not in result:
                result[img_file] = []

            # If a row contains only an image path, it's an image without annotations.
            if (x1, y1, x2, y2, class_name) == ('', '', '', '', ''):
                x1, y1, x2, y2, class_name = '0', '0', '0', '0', '0'

            x1 = parse(x1, int, 'line {}: malformed x1: {{}}'.format(line))
            y1 = parse(y1, int, 'line {}: malformed y1: {{}}'.format(line))
            x2 = parse(x2, int, 'line {}: malformed x2: {{}}'.format(line))
            y2 = parse(y2, int, 'line {}: malformed y2: {{}}'.format(line))
            
            if class_name not in classes:
                raise ValueError(f'line {line}: unknown class name: \'{class_name}\' (classes: {classes})')

            result[img_file].append({'x1': x1, 'y1': y1, 'x2': x2, 'y2': y2, 'class': class_name})
            
        except ValueError:
            raise_from(ValueError(
                f'line {line}: format should be \'img_file,x1,y1,x2,y2,class_name\' or \'img_file,,,,,\''),
                None)

    return result

In [ ]:
chosen_annotations = data_path + "all_original_training_annotations.csv"
image_data = {}
try:
    with open(chosen_annotations, 'r', newline='') as file:
        image_data = read_annotations(csv.reader(file, delimiter=','), classes=['0', '1'])
except ValueError as e:
    raise_from(ValueError('invalid CSV annotations file: {}: {}'.format(chosen_annotations, e)), None)
print(image_data)

OrderedDict([('D8C08190-E91D-4480-B229-DCA56426342B-3.png', [{'x1': 18, 'y1': 100, 'x2': 33, 'y2': 225, 'class': '1'}]), ('D8C08190-E91D-4480-B229-DCA56426342B-4.png', [{'x1': 201, 'y1': 100, 'x2': 223, 'y2': 225, 'class': '1'}]), ('D8C08190-E91D-4480-B229-DCA56426342B-5.png', [{'x1': 22, 'y1': 100, 'x2': 37, 'y2': 225, 'class': '1'}]), ('D8C08190-E91D-4480-B229-DCA56426342B-7.png', [{'x1': 206, 'y1': 100, 'x2': 238, 'y2': 225, 'class': '1'}]), ('D8C08190-E91D-4480-B229-DCA56426342B-13.png', [{'x1': 189, 'y1': 100, 'x2': 221, 'y2': 225, 'class': '1'}]), ('D8C08190-E91D-4480-B229-DCA56426342B-14.png', [{'x1': 125, 'y1': 100, 'x2': 147, 'y2': 225, 'class': '1'}]), ('D8C08190-E91D-4480-B229-DCA56426342B-18.png', [{'x1': 87, 'y1': 100, 'x2': 103, 'y2': 225, 'class': '1'}]), ('D8C08190-E91D-4480-B229-DCA56426342B-20.png', [{'x1': 71, 'y1': 100, 'x2': 102, 'y2': 225, 'class': '1'}]), ('D8C08190-E91D-4480-B229-DCA56426342B-22.png', [{'x1': 224, 'y1': 100, 'x2': 240, 'y2': 225, 'class': '1'}])

In [ ]:
output_annotations_path = data_path + "/annotations/processed_anotations.csv" 
with open(output_annotations_path, mode='w') as csv_file:
    fieldnames = ['image_name', 'x1', 'y1', 'x2', 'y2', 'class']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    writer.writeheader()
    for image in list(image_data.items()):
      writer.writerow({'image_name': image[0], 'x1': image[1][0]['x1'], 'y1': image[1][0]['y1'], 'x2': image[1][0]['x2'], 'y2': image[1][0]['y2'], 'class': image[1][0]['class']})
